# Results p. 3: Boundary Sensitivity
## Fixation Boundaries

In [1]:
import os
import copy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_sdt as ch_sdt

pio.renderers.default = "browser"

C:\Users\jonathanni\PycharmProjects\pEYES\venv\Lib\site-packages\statsmodels\tools\tools.py:6: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 2.3.0)
  import scipy.linalg


### Set Constants

In [2]:
THRESHOLD = 5       # temporal threshold for analyzing d'
METRIC = peyes.constants.D_PRIME_STR        # can also be `peyes.constants.F1_STR` or `peyes.constants.CRITERION_STR`
EVENT_NAME, EVENT_LABEL = "Fixation", 1    # EventLabelEnum.FIXATION.value

GRID_LINE_COLOR, GRID_LINE_WIDTH = "lightgray", 1
ZERO_LINE_WIDTH = 2 * GRID_LINE_WIDTH
LINE_WIDTH, ERROR_WIDTH = 2 * ZERO_LINE_WIDTH, ZERO_LINE_WIDTH
MARKER_SIZE = LINE_WIDTH * 2

FONT_FAMILY, FONT_COLOR = "Calibri", "black"
TITLE_FONT = dict(family=FONT_FAMILY, size=26, color=FONT_COLOR)
AXIS_LABEL_FONT = dict(family=FONT_FAMILY, size=23, color=FONT_COLOR)
AXIS_TICK_FONT = dict(family=FONT_FAMILY, size=20, color=FONT_COLOR)
AXIS_TITLE_STANDOFF = 2

## Load Data

In [3]:
csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=EVENT_LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics

## Multi-Threshold Figure
### (not in the paper)
This figure shows the `METRIC` values at increasing temporal thresholds: $$\Delta t = 0, 1, ..., 20$$
Each detector is shown in a different color, with error bars showing the SEM across recordings.
The left colomn show results when `GT1` (_RA_) is used as the ground truth, while the right column shows results when `GT2` (_MN_) is used.

In [4]:
W, H = 1300, 450

multi_thresholds_figure = ch_sdt.multi_channel_figure(
    csdt_metrics,
    metric=METRIC,
    yaxis_title=r"d'", show_other_gt=True,
    error_bars='std', show_err_bands=False,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    subplots_hspace=0, subplots_vspace=0,
)

multi_thresholds_figure.update_layout(
    width=W, height=H,
    title=dict(text=EVENT_NAME + "s", y=0.975, x=0.5, xanchor='center'),
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',

    # remove axis grids
    xaxis=dict(showgrid=False, zeroline=False, showline=False), yaxis=dict(showgrid=False, zeroline=False, showline=False),
    xaxis2=dict(showgrid=False, zeroline=False, showline=False), yaxis2=dict(showgrid=False, zeroline=False, showline=False),
    xaxis3=dict(showgrid=False, zeroline=False, showline=False), yaxis3=dict(showgrid=False, zeroline=False, showline=False),
    xaxis4=dict(showgrid=False, zeroline=False, showline=False), yaxis4=dict(showgrid=False, zeroline=False, showline=False),

    # move legend to bottom
    legend=dict(orientation="h", yanchor="top", xanchor="center", xref='container', yref='container', x=0.5, y=0.05),
    # showlegend=False,   # hide legend
    margin=dict(l=10, r=0, b=10, t=0, pad=0),
)
multi_thresholds_figure.layout.annotations = []   # remove subtitles

# FIG_ID, PANEL_ID, IS_SUPP = 6, '', True
# save_fig(multi_thresholds_figure, FIG_ID, PANEL_ID, f"{TITLE.lower()}discrimination_multi_threshold-{METRIC.lower()}", IS_SUPP)

# multi_thresholds_figure.show()

## Statistical Analysis
### Onsets

In [5]:
onset_statistics, onset_pvalues, onset_nemenyi, onset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "onset", THRESHOLD, [GT1, GT2]
)
onset_post_hoc = ch_sdt.post_hoc_table(
    onset_nemenyi, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat([onset_statistics, onset_pvalues, onset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(onset_post_hoc)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  63.491613  8.763098e-12    True
                 RA  92.342391  9.876963e-18    True
d_prime          MN  65.969032  2.734732e-12    True
                 RA  84.625000  3.947687e-16    True
f1               MN  62.093023  1.688984e-11    True
                 RA  64.666667  5.045799e-12    True
false_alarm_rate MN  38.875000  7.572949e-07    True
                 RA  44.695814  5.378847e-08    True
precision        MN  49.803109  5.148264e-09    True
                 RA  71.650591  1.874366e-13    True
recall           MN  68.952000  6.705755e-13    True
                 RA  90.106742  2.879170e-17    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.      **      **    n.s.    n.s.     n.s.
ivvt     MN  0.8729      --      **      **    n.s.    n.s.     n.s.
         RA  0.9447      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.1324  0.0012      --    n.s.     ***       *       **
         RA  0.0056  0.0000      --    n.s.     ***      **      ***
idvt     MN  0.1539  0.0015  1.0000      --     ***       *       **
         RA  0.0066  0.0000  1.0000      --     ***      **      ***
engbert  MN  0.3409  0.9826  0.0000  0.0000      --    n.s.     n.s.
         RA  0.4292  0.9726  0.0000  0.0000      --    n.s.     n.s.
nh       MN  0.9923  0.9983  0.0127  0.0160  0.8163      --     n.s.
         RA  0.9998  0.9936  0.0011  0.0013  0.6809      --     n.s.
remodnav MN  0.9598  1.0000  0.0042  0.0054  0.9277  1.0000       --
         RA  0.9979  0.9991  0.0004  0.0005  0.8085  1.0000       --

### Offsets

In [6]:
offset_statistics, offset_pvalues, offset_nemenyi, offset_Ns = ch_sdt.friedman_nemenyi(
    csdt_metrics, "offset", THRESHOLD, [GT1, GT2]
)
offset_post_hoc = ch_sdt.post_hoc_table(
    offset_nemenyi, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat([offset_statistics, offset_pvalues, offset_pvalues <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(offset_post_hoc)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  67.891473  1.105778e-12    True
                 RA  98.420671  5.357366e-19    True
d_prime          MN  59.085271  6.903108e-11    True
                 RA  79.425204  4.695975e-15    True
f1               MN  48.830530  8.061506e-09    True
                 RA  62.627380  1.314607e-11    True
false_alarm_rate MN  29.642857  4.595452e-05    True
                 RA  49.145756  6.971519e-09    True
precision        MN  35.356021  3.676308e-06    True
                 RA  38.689844  8.232601e-07    True
recall           MN  67.799186  1.154944e-12    True
                 RA  98.429241  5.335369e-19    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       *    n.s.     ***     n.s.
         RA      --    n.s.      **      **    n.s.     ***        *
ivvt     MN  1.0000      --       *       †    n.s.     ***     n.s.
         RA  0.9997      --       *       *    n.s.     ***     n.s.
idt      MN  0.0165  0.0401      --    n.s.      **    n.s.     n.s.
         RA  0.0030  0.0157      --    n.s.     ***    n.s.     n.s.
idvt     MN  0.0313  0.0703  1.0000      --      **    n.s.     n.s.
         RA  0.0046  0.0228  1.0000      --     ***    n.s.     n.s.
engbert  MN  0.9996  0.9948  0.0029  0.0063      --     ***        *
         RA  0.9806  0.8774  0.0000  0.0001      --     ***      ***
nh       MN  0.0001  0.0005  0.9536  0.9017  0.0000      --     n.s.
         RA  0.0000  0.0002  0.9612  0.9377  0.0000      --     n.s.
remodnav MN  0.0961  0.1836  0.9985  0.9999  0.0252  0.7186       --
         RA  0.0299  0.1048  0.9977  0.9992  0.0008  0.7140       --

## Figure: Distribution @ Threshold
### (not in the paper)
This figure shows the distribution of `METRIC` at the predefined temporal `THRESHOLD`, for each detector across recordings.
The right-side of each violin is w.r.t. `GT1` (_RA_) as the ground truth annotator, and the left-side is w.r.t. `GT2` (_MN_).

In [7]:
W, H = 600, 450

onset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.ONSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

offset_distribution_figure = ch_sdt.single_threshold_figure(
    csdt_metrics.loc[(slice(None), [peyes.constants.D_PRIME_STR, peyes.constants.F1_STR], slice(None)), :],
    peyes.constants.OFFSET_STR,
    THRESHOLD,
    GT1,
    gt2=GT2,
    show_other_gt=True,
    share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
)

######################################
## COMBINE ONSET AND OFFSET FIGURES ##
######################################

discriminability_figure = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=[f"{EVENT_NAME} {peyes.constants.ONSET_STR}", f"{EVENT_NAME} {peyes.constants.OFFSET_STR}"]
)

# copy onset d-prime violins into new figure
for tr in onset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=1, col=1)

# copy offset d-prime violins into new figure
for tr in offset_distribution_figure.data:
    if tr['scalegroup'] != 'd_prime':
        # ignore non d-prime violins
        continue
    new_tr = copy.deepcopy(tr)
    new_tr['width'] = 0.8   # make violins wider so there's less space between them
    discriminability_figure.add_trace(trace=new_tr, row=2, col=1)

discriminability_figure.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    yaxis2=dict(showgrid=False, zeroline=False, showline=False, tickfont=dict(size=14)),
    xaxis2=dict(showgrid=False, tickfont=dict(size=14), tickangle=0),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
)
# discriminability_figure.layout.annotations = []   # remove subtitles

# FIG_ID, IS_SUPP = 5, False
# save_fig(discriminability_figure, FIG_ID, 'left', 'fixation-discriminability', IS_SUPP)

# discriminability_figure.show()

## Final Figure
### (Shown in the paper)
- Combine Multi-Threshold line-plots with Single-Threshold violin-plots
- use only one GT annotator

In [26]:
WIDTH, HEIGHT = 1600, 900
COLUMN_TITLES = [f"{EVENT_NAME} {peyes.constants.ONSET_STR.capitalize()}s", f"{EVENT_NAME} {peyes.constants.OFFSET_STR.capitalize()}s",]


def _convert_line_traces(line_fig: go.Figure, new_fig: go.Figure, gt: str) -> go.Figure:
    if gt == "RA":
        other = "MN"
        allowed_yaxes = ["y", "y3"]
    elif gt == "MN":
        other = "RA"
        allowed_yaxes = ["y2", "y4"]
    else:
        raise ValueError("Unknown GT: {}".format(gt))

    for trace in line_fig.data:
        if not trace['yaxis'] in allowed_yaxes:
            # ignore traces that don't use the selected GT
            continue
        new_tr = copy.deepcopy(trace)
        new_tr["showlegend"] = False
        new_tr["line_width"] = LINE_WIDTH
        new_tr["error_y"]["thickness"] = ERROR_WIDTH
        new_tr["marker"]["size"] = LINE_WIDTH * 2

        if new_tr["name"] == "Other GT":
            new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
            new_tr["marker"]["color"] = new_tr["line"]["color"] = "gray"
        elif new_tr["name"].startswith("i"):
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].replace("i", "I-").upper()
        elif new_tr["name"] == "remodnav":
            new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["name"] = new_tr["legendgroup"] = new_tr["name"].upper()

        # add the trace to the correct subplot
        is_onset = trace['yaxis'] in ['y', 'y2']   # onset traces are in top row
        new_fig.add_trace(new_tr, row=1, col=1 if is_onset else 2)
    return new_fig


def _convert_violin_traces(violin_fig, new_fig, gt: str) -> go.Figure:
    other = "MN" if gt == "RA" else "RA"
    for trace in violin_fig.data:
        if trace['scalegroup'] != METRIC:
            # ignore non `METRIC` traces
            continue
        if not trace["name"].startswith(gt):
            # ignore traces that don't use the selected GT
            continue
        new_tr = copy.deepcopy(trace)
        new_tr['scalegroup'] = "VIOLINS"
        new_tr["showlegend"] = False

        # convert from violin plot to ridge plot by assigning the `y0` prop and removing `x0` prop
        if new_tr["x0"] == "Other GT":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = f"Ann. {other}"
        elif new_tr["x0"].startswith("i"):
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].replace("i", "I-").upper()
        elif new_tr["x0"] == "remodnav":
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = "REMoDNaV"
        else:
            new_tr["y0"] = new_tr["name"] = new_tr["legendgroup"] = new_tr["x0"].upper()
        new_tr['x'] = new_tr['y']

        # reset irrelevant props
        new_tr['y'] = new_tr['x0'] = None
        new_tr["showlegend"] = new_tr['points'] = False
        new_tr["side"] = "positive"
        new_tr["width"] = 1.8
        new_tr["box"] = dict(visible=False, width=0.95, line=dict(width=1))
        new_tr["meanline"] = dict(visible=True, width=3, color='lightgray')
        new_tr["opacity"] = 1
        new_tr["visible"] = True

        # add the trace to the correct subplot
        is_onset = trace['yaxis'] == 'y'
        new_fig.add_trace(new_tr, row=2, col=1 if is_onset else 2)
    return new_fig


def convert_figs(line_fig, violin_fig, gt: str) -> go.Figure:
    # copy traces to new figure
    new_fig = make_subplots(
        rows=len(COLUMN_TITLES), cols=len(COLUMN_TITLES),
        shared_xaxes=False, shared_yaxes=True,
        vertical_spacing=0.12, horizontal_spacing=0.02,
        column_titles=COLUMN_TITLES,
    )
    _convert_line_traces(line_fig, new_fig, gt)
    _convert_violin_traces(violin_fig, new_fig, gt)

    # update legends
    new_fig.for_each_trace(lambda tr: tr.update(showlegend=tr['yaxis'] == 'y'))  # legend only shows one set of lines

    # update subtitles
    new_fig.for_each_annotation(lambda ann: ann.update(text=f"<b>{ann.text}</b>", font=TITLE_FONT, textangle=0, ))

    # add red rectangles in the top plots to to highlight the bottom subplots' origin
    for c in range(len(COLUMN_TITLES)):
        if gt=="RA":
            y0 = -0.9 if c == 0 else 0.9
            y1 = 4.1 if c == 0 else 5.5
        else:
            y0 = -0.65 if c == 0 else 0.95
            y1 = 3.65 if c == 0 else 5.3
        new_fig.add_shape(
            row=1, col=c + 1,
            type="rect",
            x0=THRESHOLD-0.5, x1=THRESHOLD+0.5,
            y0=y0, y1=y1,
            line_color="red", line_width=LINE_WIDTH,
        )

    # add axis labels
    for r in range(len(COLUMN_TITLES)):
        xaxis_title = "Δt (samples)" if r == 0 else r"$d'$"
        yaxis_title = r"$d'$" if r == 0 else "Detector"
        for c in range(len(COLUMN_TITLES)):
            new_fig.update_xaxes(
                row=r + 1, col=c + 1,
                title=dict(text=xaxis_title, font=AXIS_LABEL_FONT, standoff=AXIS_TITLE_STANDOFF),
                showline=False,
                showgrid=r == 0, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
                zeroline=r==0, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
                tickfont=AXIS_TICK_FONT,
            )
            new_fig.update_yaxes(
                row=r + 1, col=c + 1,
                title=dict(text=yaxis_title if c == 0 else "", font=AXIS_LABEL_FONT, standoff=AXIS_TITLE_STANDOFF),
                showline=False,
                showgrid=True, gridcolor=GRID_LINE_COLOR, gridwidth=GRID_LINE_WIDTH,
                zeroline=r==0, zerolinecolor=GRID_LINE_COLOR, zerolinewidth=ZERO_LINE_WIDTH,
                showticklabels=r==0 and c==0, tickfont=AXIS_TICK_FONT, tickangle=0 if r==0 else 30,
            )

    # add row annotations "A" and "B"
    for ann in ["A", "B"]:
        new_fig.add_annotation(
            text=f"<b>{ann}</b>", font={**TITLE_FONT, **dict(size=36)}, showarrow=False,
            xref="paper", yref="paper", xanchor="right", yanchor="top", x=0, y=1.05 if ann == "A" else 0.46
        )


    new_fig.update_layout(
        font_family=FONT_FAMILY,
        width=WIDTH, height=HEIGHT,
        paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
        margin=dict(l=0, r=0, t=30, b=0, pad=0),
        legend=dict(
            orientation="h", bgcolor='rgba(0, 0, 0, 0)',
            yanchor="top", y=-0.06, xanchor="center", x=0.5,
            font=AXIS_TICK_FONT, itemwidth=90,
        ),
    )
    return new_fig

In [30]:
GT = "RA"
NAME = f"fig4-{GT}"

fig = convert_figs(multi_thresholds_figure, discriminability_figure, GT)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"))
fig.show()

In [27]:
GT = "MN"
NAME = f"supp_fig_G1-{GT}"

fig = convert_figs(multi_thresholds_figure, discriminability_figure, GT)

fig.write_image(os.path.join(FIGURES_DIR, f"{NAME}.png"), scale=3)
# fig.write_json(os.path.join(FIGURES_DIR, f"{NAME}.json"))
fig.show()

## Repeat Analysis
### Repeating the analysis with subset of image-stimulus trials that were recorded @ 500Hz

In [11]:
csdt_metrics = ch_sdt.load(
    dataset_name=DATASET_NAME,
    output_dir=PROCESSED_DATA_DIR,
    label=EVENT_LABEL,
    stimulus_type=STIMULUS_TYPE,
    channel_type=None,
)
csdt_metrics.drop(index=['P', 'PP', 'N', 'TP'], level=peyes.constants.METRIC_STR, inplace=True)    # Remove unused metrics
csdt_subset = csdt_metrics.drop(columns=NON_500HZ_TRIALS, level=peyes.constants.TRIAL_ID_STR)

### Onsets

In [12]:
onset_statistics_subset, onset_pvalues_subset, onset_nemenyi_subset, onset_Ns_subset = ch_sdt.friedman_nemenyi(
    csdt_subset, "onset", THRESHOLD, [GT1, GT2]
)
onset_post_hoc_subset = ch_sdt.post_hoc_table(
    onset_nemenyi_subset, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat(
        [onset_statistics_subset, onset_pvalues_subset, onset_pvalues_subset <= ALPHA],
        axis=1, keys=['Q', 'p', 'is_sig']
    ).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(onset_post_hoc_subset)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  57.722892  1.304103e-10    True
                 RA  75.936652  2.461756e-14    True
d_prime          MN  57.867470  1.219031e-10    True
                 RA  68.076923  1.013206e-12    True
f1               MN  53.746606  8.298092e-10    True
                 RA  57.685611  1.326983e-10    True
false_alarm_rate MN  37.062027  1.712536e-06    True
                 RA  39.951220  4.656805e-07    True
precision        MN  41.818457  1.996942e-07    True
                 RA  56.604545  2.196632e-10    True
recall           MN  62.040562  1.731039e-11    True
                 RA  74.856810  4.107891e-14    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.    n.s.    n.s.    n.s.    n.s.     n.s.
         RA      --    n.s.       *       *    n.s.    n.s.     n.s.
ivvt     MN  0.9130      --      **      **    n.s.    n.s.     n.s.
         RA  0.9572      --     ***     ***    n.s.    n.s.     n.s.
idt      MN  0.1807  0.0034      --    n.s.     ***    n.s.       **
         RA  0.0234  0.0003      --    n.s.     ***       *      ***
idvt     MN  0.2113  0.0045  1.0000      --     ***    n.s.       **
         RA  0.0282  0.0003  1.0000      --     ***       *       **
engbert  MN  0.5496  0.9961  0.0001  0.0002      --    n.s.     n.s.
         RA  0.6207  0.9935  0.0000  0.0000      --    n.s.     n.s.
nh       MN  0.9999  0.9818  0.0775  0.0942  0.7626      --     n.s.
         RA  1.0000  0.9315  0.0337  0.0402  0.5478      --     n.s.
remodnav MN  0.9586  1.0000  0.0070  0.0092  0.9859  0.9945       --
         RA  0.9884  1.0000  0.0008  0.0011  0.9716  0.9776       --

### Offsets

In [13]:
offset_statistics_subset, offset_pvalues_subset, offset_nemenyi_subset, offset_Ns_subset = ch_sdt.friedman_nemenyi(
    csdt_subset, "offset", THRESHOLD, [GT1, GT2]
)
offset_post_hoc_subset = ch_sdt.post_hoc_table(
    offset_nemenyi_subset, METRIC, [GT1, GT2], ALPHA, marginal_alpha=MARGINAL_ALPHA
)

print("Friedman test results:")
display(
    pd.concat(
        [offset_statistics_subset, offset_pvalues_subset, offset_pvalues_subset <= ALPHA], axis=1, keys=['Q', 'p', 'is_sig']
    ).stack(1, future_stack=True)
)

print("\n#################################\n")
print(f"Tukey-HSD post-hoc test results for metric {METRIC}:")
display(offset_post_hoc_subset)

Friedman test results:


Q             p  is_sig
metric           gt                                 
criterion        MN  60.841629  3.035657e-11    True
                 RA  82.993197  8.593596e-16    True
d_prime          MN  55.918552  3.023456e-10    True
                 RA  71.986395  1.599131e-13    True
f1               MN  46.812689  2.039190e-08    True
                 RA  59.346939  6.108480e-11    True
false_alarm_rate MN  29.761103  4.363937e-05    True
                 RA  40.321101  3.938918e-07    True
precision        MN  31.400000  2.125857e-05    True
                 RA  32.214857  1.484067e-05    True
recall           MN  61.876777  1.869176e-11    True
                 RA  85.121034  3.115926e-16    True


#################################

Tukey-HSD post-hoc test results for metric d_prime:


pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --    n.s.       *       †    n.s.     ***     n.s.
         RA      --    n.s.       *       *    n.s.     ***     n.s.
ivvt     MN  0.9993      --    n.s.    n.s.    n.s.     ***     n.s.
         RA  0.9958      --    n.s.    n.s.    n.s.     ***     n.s.
idt      MN  0.0484  0.1778      --    n.s.       *    n.s.     n.s.
         RA  0.0172  0.1335      --    n.s.      **    n.s.     n.s.
idvt     MN  0.0612  0.2113  1.0000      --       *    n.s.     n.s.
         RA  0.0223  0.1595  1.0000      --      **    n.s.     n.s.
engbert  MN  1.0000  0.9957  0.0261  0.0338      --     ***     n.s.
         RA  0.9984  0.9044  0.0017  0.0024      --     ***        *
nh       MN  0.0000  0.0003  0.6721  0.6211  0.0000      --     n.s.
         RA  0.0000  0.0000  0.4825  0.4336  0.0000      --     n.s.
remodnav MN  0.2097  0.4921  0.9985  0.9994  0.1347  0.3093       --
         RA  0.1928  0.5945  0.9869  0.9920  0.0417  0.0865       --